<a href="https://colab.research.google.com/github/Dexheimer/cotacao_ceasa/blob/main/Precos_Ceasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurações Iniciais
Criar um arquivo ou célula para as bibliotecas, configurações e funções auxiliares.

In [31]:
# Instalar as bibliotecas necessárias
!pip install pdfplumber requests beautifulsoup4 pandas

# Importar as bibliotecas
import requests
import pandas as pd
import pdfplumber
import os
from bs4 import BeautifulSoup
from datetime import datetime

# configurações iniciais

base_url = "https://www.rj.gov.br/ceasa/node/388"
base_prefix = "https://www.rj.gov.br"
output_dir = "ceasa_pdfs"
csv_output = "cotacoes_ceasa.csv"



# 1.1 Criar diretório

In [32]:
# Função para Criar Diretórios

def create_directory(path):
    """Cria um diretório se ele não existir."""
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Diretório criado: {path}")
    else:
        print(f"Diretório já existe: {path}")

In [33]:
# Testar a criação de diretório

test_dir = "teste_diretorio"
create_directory(test_dir)
assert os.path.exists(test_dir), "O diretório não foi criado corretamente."
print("Teste de criação de diretório concluído com sucesso!")
os.rmdir(test_dir)  # Limpar após o teste

Diretório criado: teste_diretorio
Teste de criação de diretório concluído com sucesso!


# 2. Coletar Links dos Meses
E testar os links para os meses

In [34]:
def get_month_links(base_url, base_prefix):
    """
    Acessa a página principal e retorna uma lista de dicionários com links e textos dos meses.
    URLs relativas são ajustadas para incluir o prefixo completo.
    """
    response = requests.get(base_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Focar na div específica onde os links estão localizados
    section = soup.find("div", {"property": "schema:text"})

    # Verificar se a seção foi encontrada
    if section:
        links = section.find_all("a", href=True)
        month_links = []

        for link in links:
            url = link['href']
            # Adicionar o prefixo base para URLs relativas
            if not url.startswith("http"):
                url = f"{base_prefix}{url}"
            month_links.append({"url": url, "text": link.text.strip()})

        return month_links
    else:
        print("Seção com links não encontrada.")
        return []



In [37]:
# Testar a coleta de links
month_links = get_month_links(base_url, base_prefix)
assert isinstance(month_links, list), "Os links dos meses não foram coletados corretamente."
assert all("url" in link and "text" in link for link in month_links), "Estrutura dos links está incorreta."
assert all(link["url"].startswith("http") for link in month_links), "Algumas URLs não foram corrigidas."
print(f"Teste de coleta de links dos meses concluído! {(month_links)} links encontrados.")


Teste de coleta de links dos meses concluído! [{'url': 'https://www.rj.gov.br/ceasa/node/495', 'text': 'Dezembro'}, {'url': 'https://www.rj.gov.br/ceasa/node/491', 'text': 'Novembro'}, {'url': 'https://www.rj.gov.br/ceasa/node/452', 'text': 'Outubro'}, {'url': 'https://www.rj.gov.br/ceasa/node/446', 'text': 'Setembro'}, {'url': 'https://www.rj.gov.br/ceasa/node/436', 'text': 'Agosto'}, {'url': 'https://www.rj.gov.br/ceasa/node/425', 'text': 'Julho'}, {'url': 'https://www.rj.gov.br/ceasa/node/403', 'text': 'Junho'}, {'url': 'https://www.rj.gov.br/ceasa/node/396', 'text': 'Maio'}, {'url': 'https://www.rj.gov.br/ceasa/node/395', 'text': 'Abril'}, {'url': 'https://www.rj.gov.br/ceasa/node/392', 'text': 'Março'}, {'url': 'https://www.rj.gov.br/ceasa/node/391', 'text': 'Fevereiro'}, {'url': 'https://www.rj.gov.br/ceasa/node/387', 'text': 'Janeiro'}] links encontrados.


# 4. Função para Extrair Links de PDFs

In [43]:
def get_daily_pdf_links(month_url, month_name):
    """
    Acessa uma página do mês e retorna os links para os PDFs diários associados ao mês.
    """
    response = requests.get(month_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todos os links para arquivos PDF
    pdf_links = [
        {"url": link['href'], "month": month_name}
        for link in soup.find_all("a", href=True)
        if link['href'].endswith(".pdf")
    ]
    return pdf_links



In [44]:
# Testar a função de extração de links de PDFs
test_month_url = "https://www.rj.gov.br/ceasa/node/491"  # Exemplo para Novembro
test_month_name = "Novembro"

pdf_links = get_daily_pdf_links(test_month_url, test_month_name)

# Validar os resultados
assert isinstance(pdf_links, list), "A função não retornou uma lista."
assert all("url" in link and "month" in link for link in pdf_links), "Estrutura dos links está incorreta."
print(f"Teste de extração de links de PDFs concluído! {len(pdf_links)} PDFs encontrados para {test_month_name}.")


Teste de extração de links de PDFs concluído! 17 PDFs encontrados para Novembro.
